In [ ]:
from train_tf2 import Trainer
import matplotlib.pyplot as plt
import numpy as np

train_filenames = "../data/training_flat_256_3d.tfrecords"
val_filenames = "../data/validation_flat_256_3d.tfrecords"
#Trainer(model='FF').run(train_filenames, val_filenames, x_slice=127)
Trainer(model='CNN').run(train_filenames, val_filenames, x_slice=127)

Epoch  #1 |                   |   1% Cost: 3.3962, Avg. Accuracy: 0.3000, Current Accuracy: 0.40, elapsed time: 88s

In [ ]:
def slice_accuracy(dim="x"):
    accuracies = []
    range_ = np.arange(50, 206, 15)
    for slice_ in range_:
        if dim == "x":
            accuracies.append(Trainer().run(train_filenames, val_filenames, x_slice=slice_))
        elif dim == "y":
            accuracies.append(Trainer().run(train_filenames, val_filenames, y_slice=slice_))
        elif dim == "z":
            accuracies.append(Trainer().run(train_filenames, val_filenames, z_slice=slice_))
    return accuracies
    
accuracies = slice_accuracy(dim="x")    
plt.plot(range_, accuracies)
plt.title("Accuracies for 2d slices in x-dimension")
plt.xlabel("pixel")
plt.ylabel("accuracy")
plt.show()

Epoch  #1 |################## |  93% Cost 1.6135, Avg. Accuracy 0.2047, Current Accuracy: 0.20

In [1]:
import tensorflow as tf
name = "../data/training_flat_256_3d.tfrecords"
def _parse_function(example_proto):
    features = {"image": tf.io.FixedLenFeature([256, 256, 256], tf.float32),
              "label": tf.io.FixedLenFeature((), tf.int64),
               'name': tf.io.FixedLenFeature((), tf.string, default_value='')}
    parsed_features = tf.io.parse_single_example(serialized=example_proto, features=features)
    parsed_features["image"]= tf.reshape(parsed_features["image"],[-1]) 
    parsed_features["label"]=tf.one_hot(parsed_features["label"],5)
    return parsed_features["image"], parsed_features["label"], parsed_features["name"]

    return parsed_features["image"]
batchsize = 1
dataset = tf.data.TFRecordDataset(name).map(_parse_function)#.shuffle(
                    #buffer_size=10).batch(batchsize)
count=0
for image, label, name in dataset:
    count +=1
    #im_ = tf.reshape(image, [batchsize, 256, 256, 256])
    #label = tf.reshape(label, [batchsize, 5])
    #name = tf.reshape(name, [batchsize, -1])
    
    #print(im_[1, :, :, :].shape)
    #print(label)
    print("\r" + str(count), end="")
# Count the records
#records_n = sum(1 for record in dataset)

#print("records_n = {}".format(records_n))

12975

In [56]:
class model():
    
    def cost(self, X, y):
    #activations, logits = self.model(X, y, batch_size)
        return tf.reduce_mean(input_tensor=tf.nn.softmax_cross_entropy_with_logits(logits=X, labels=y))

    def forward(self, X):
        self.w_fc = tf.Variable(tf.random.truncated_normal(shape=[100, 5], dtype=tf.float64, stddev=0.1), name='w_{0}'.format("hi"))
        self.b_fc = tf.Variable(tf.constant(0.0, shape=[5], dtype=tf.float64), name='b_{0}'.format("hi"))
        self.variables = [self.w_fc, self.b_fc]
        return tf.nn.relu(tf.nn.bias_add(tf.matmul(X, self.w_fc), self.b_fc), name="hi")
    
    def backward(self, X_train, y_train):
        optimizer = tf.compat.v1.train.AdamOptimizer(0.00001)
        with tf.GradientTape() as tape:
            predicted = self.forward(X_train)
            current_loss = self.cost(predicted, y_train)
        grads = tape.gradient(current_loss, self.variables)
        optimizer.apply_gradients(zip(grads, self.variables))



In [57]:
import tensorflow as tf
import numpy as np
#optimizer = tf.compat.v1.train.AdamOptimizer(0.00001)
x = np.random.rand(5, 100)
y = (np.sum(x, axis=1)/10).astype(int)

#w_fc = tf.Variable(tf.random.truncated_normal(shape=[100, 5], dtype=tf.float64, stddev=0.1), name='w_{0}'.format("hi"))
#tf.matmul(x, w_fc)
model_default = model()

preds = model_default.forward(x)

#loss_total = loss_total + model_default.loss(preds, y)

model_default.backward(x, y)